In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [2]:
df_model = pd.read_pickle('../data/pickle_emotion/df_model_en_.p')

In [3]:
df_model.head()

count_words  count_mentions  count_hashtags  count_capital_words  \
0  6            0               3               0                     
1  8            0               2               0                     
2  16           0               3               0                     
3  11           0               3               0                     
4  16           0               4               0                     

   count_excl_quest_marks  count_urls  count_emojis sentiment  \
0  0                       0           0             fear       
1  1                       0           0             anger      
2  6                       0           3             anger      
3  0                       0           0             anger      
4  1                       0           0             joy        

                                                                                                      origin_text  \
0  Now #India is #afraid of #bad .                                                                                  
1  He's seriously so frustrating sometimes! (╯°□°）╯︵ ┻━┻ #ugh #raging                                               
2  How in the world did Nicole beat Paul?!?! #terrible #bb18 #BBFinale What was the jury thinking?? 😳😳😳             
3  Yeah, no. I don' mind tweetin' that twice. #snap #blm #likeThat                                                  
4  What's the most important thing you need to accomplish today? #onlinemarketing #solopreneur #start up #success   

                                                                        clean_text  
0  india afraid bad                                                                 
1  seriously frustrating sometimes ╯°□°）╯︵ ┻━┻ ugh raging                           
2  world nicole beat paul terrible bb bbfinale jury thinking 😳😳😳                    
3  yeah no mind tweetin twice snap blm likethat                                     
4  important thing need accomplish today onlinemarketing solopreneur start success

In [4]:
#replace class name with number
df_model['sentiment'][df_model.sentiment == 'anger'] = 1
df_model['sentiment'][df_model.sentiment == 'anticipation'] = 2
df_model['sentiment'][df_model.sentiment == 'disgust'] = 3
df_model['sentiment'][df_model.sentiment == 'fear'] = 4
df_model['sentiment'][df_model.sentiment == 'joy'] = 5
df_model['sentiment'][df_model.sentiment == 'love'] = 6
df_model['sentiment'][df_model.sentiment == 'optimism'] = 7
df_model['sentiment'][df_model.sentiment == 'pessimism'] = 8
df_model['sentiment'][df_model.sentiment == 'sadness'] = 9
df_model['sentiment'][df_model.sentiment == 'surprise'] = 10
df_model['sentiment'][df_model.sentiment == 'trust'] = 11
df_model.head()

count_words  count_mentions  count_hashtags  count_capital_words  \
0  6            0               3               0                     
1  8            0               2               0                     
2  16           0               3               0                     
3  11           0               3               0                     
4  16           0               4               0                     

   count_excl_quest_marks  count_urls  count_emojis sentiment  \
0  0                       0           0             4          
1  1                       0           0             1          
2  6                       0           3             1          
3  0                       0           0             1          
4  1                       0           0             5          

                                                                                                      origin_text  \
0  Now #India is #afraid of #bad .                                                                                  
1  He's seriously so frustrating sometimes! (╯°□°）╯︵ ┻━┻ #ugh #raging                                               
2  How in the world did Nicole beat Paul?!?! #terrible #bb18 #BBFinale What was the jury thinking?? 😳😳😳             
3  Yeah, no. I don' mind tweetin' that twice. #snap #blm #likeThat                                                  
4  What's the most important thing you need to accomplish today? #onlinemarketing #solopreneur #start up #success   

                                                                        clean_text  
0  india afraid bad                                                                 
1  seriously frustrating sometimes ╯°□°）╯︵ ┻━┻ ugh raging                           
2  world nicole beat paul terrible bb bbfinale jury thinking 😳😳😳                    
3  yeah no mind tweetin twice snap blm likethat                                     
4  important thing need accomplish today onlinemarketing solopreneur start success

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [6]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [7]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    

    features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                            , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [8]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for RandomForestRegressor
parameters_randomforest = {
    'clf__n_estimators'      : (100,120,130),
    'clf__max_depth'         : (8, 10, 12)
}


In [9]:
rfr = RandomForestRegressor()

In [10]:
countvect = CountVectorizer()

In [ ]:
best_rfr_countvect = grid_vect(rfr, parameters_randomforest, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__max_depth': (8, 10, 12),
 'clf__n_estimators': (100, 120, 130),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [36]:
tfidfvect = TfidfVectorizer()

In [37]:
best_rfr_tfidf = grid_vect(rfr, parameters_randomforest, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  5.0min finished


done in 308.657s

Best CV score: 0.746
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.5
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.756


Classification Report Test Data
              precision    recall  f1-score   support

    negative       0.80      0.76      0.78      1333
     neutral       0.73      0.72      0.73      1884
    positive       0.75      0.79      0.77      1730

   micro avg       0.76      0.76      0.76      4947
   macro avg       0.76      0.76      0.76      4947
weighted avg       0.76      0.76      0.76      4947

all results
0.684291 (0.004338) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.736714 (0.001216) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 2)}
0.687099 (0.004

In [19]:
#max_df: 0.25 or maximum document frequency of 25%.
#min_df: 2 or the words need to appear in at least 2 tweets
#ngram_range: (1, 2), both single words as bi-grams are used
#clf__alpha: 0.75
textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.75, min_df=2, ngram_range=(1,2)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', RandomForestRegressor(n_estimators=100,max_depth=12))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)

In [20]:
df_model_pos = pd.read_pickle('df_model_pos.p')
best_model.predict(df_model_pos).tolist()

[2.0561716146305566, 1.8334043904426147, 1.8651327465696823]

In [21]:
df_model_neg = pd.read_pickle('df_model_neg.p')
best_model.predict(df_model_neg).tolist()

[2.708841736214774, 2.3931210593563303, 2.5858595323939437, 1.966987629635001]